In [ ]:
import os
import pandas as pd

def size_without_header(file_path):
    # print("opening", file_path)
        # Read the first lines of the file until we reach some non-text (binary) data
    try:
        with open(file_path, "rb") as file:
            # print("opened", file_path)
            # Read line by line until an exception is thrown or the end of file is reached
            header = ""
            
            while True:
                line = file.readline()
                # print(line)
                if line == "":
                    break
                header += line.decode("utf-8")
            
    
    except Exception as e:
        pass
        
        # Now we know the header, we can get the size of the file
    with open(file_path, "rb") as file:
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        # Subtract the size of the header from the file size
        file_size -= len(header)
    return file_size

data_root = '/media/andrew/One Touch/HMNET_ADDER_data'
gt_directory_path = '/media/andrew/One Touch/HMNET2/experiments/detection/data/gen1/source/detection_dataset_duration_60s_ratio_1.0/test'

total_count = 0
df = pd.DataFrame(columns=['filename', 'crf', 'gt_bytes', 'shrunk_bytes', 'pct_shrunk'])

for i in [0,3,6,9]:
    # look in ./gen${i} and get the file list
    input_directory_path = gt_directory_path
    output_directory_path = os.path.join(data_root, f'gen{i}_dat')
    os.makedirs(output_directory_path, exist_ok=True)
    
    file_list = os.listdir(input_directory_path)
    file_list = [filename for filename in file_list if filename.endswith('.dat')]
    
    for filename in file_list:
        
        # get filename without extension
        filename_out = filename.split('.')[0]+'.dat'
            
        input_file_path = os.path.join(input_directory_path, filename)
        output_file_path = os.path.join(output_directory_path, filename_out)
        
        if filename_out in already_done_files and os.path.getsize(output_file_path) > 100:
            total_count = total_count + 1
            
            crf = i
            gt_bytes = size_without_header(input_file_path)
            shrunk_bytes = size_without_header(output_file_path)
            pct_shrunk = (gt_bytes - shrunk_bytes) / gt_bytes * 100
    
            # Create a new row and append it to the DataFrame
            new_row = pd.DataFrame({'filename': [filename_out], 'crf': [crf], 'gt_bytes': [gt_bytes], 'shrunk_bytes': [shrunk_bytes], 'pct_shrunk': [pct_shrunk]})
            df = pd.concat([df, new_row], ignore_index=True)
    
    print("CRF", i, "done")
    
print("Total count", total_count)
print(df)
df.to_csv('shrunk_files.csv', index=False)